In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from requests.structures import CaseInsensitiveDict

# List of cities used in search.
CITY_LIST = [('Los%20Angeles', 'CA'),
             ('Miami', 'FL'),
             ('Chicago', 'IL'),
             ('Houston', 'TX'),
             ('Phoenix', 'AZ'),
             ('Dallas', 'TX'),
             ('Fort%20Lauderdale', 'FL'),
             ('Las%20Vegas', 'NV'),
             ('New%20York%20City', 'NY'),
             ('Riverside', 'CA'),
             ('San%20Bernardino', 'CA'),
             ('Fort%20Worth', 'TX'),
             ('San%20Diego', 'CA'),
             ('El%20Paso', 'TX'),
             ('Denver', 'CO'),
             ('San%20Francisco', 'CA'),
             ('San%20Jose', 'CA'),
             ('Portland', 'OR'),
             ('Seattle', 'WA')]

# List of doctor specialties used in search. Does not limit the types to be added to the spreadsheet,
# only provides the initial search query.
JOB_TYPE = ['Internal%20Medicine', 'Family%20Medicine', 'Pediatrics', 'Obstetrics%20%26%20Gynecology', 'Psychiatry',
            'Infectious%20Disease', 'Dermatology', 'Dentistry', 'Opthalmology']


# Supplemental function to handle comment data retrieval. Keeps things slightly neater.
# Returns comments for given provider ID in json format as provided by the website.
def get_provider_comments(id):
    comment_url = 'https://www.healthgrades.com/api4/providerprofile/comments'
    payload = f"currentPage=1&includeAllAnswers=true&perPage=1000&pwid={id}&sortOption=1"
    headers = CaseInsensitiveDict()
    headers["Content-Type"] = "application/x-www-form-urlencoded"
    comments = requests.post(comment_url, headers=headers, data=payload).json()
    return comments

# Not the most readable, due for a refactor if code needs to be used for anything outside of it's current (narrow) scope.
# Function to retrieve and return all data for a given provider.

# Number to list --> how many providers per specialty type
# Distance --> radius (miles) to search for providers (default 50 miles)
# Ignore_below --> how many comments a provider needs to be considered valid (default 10 comments)
def establish_provider_data(number_to_list, distance=50, ignore_below=10):
    Physician_name = []
    Physician_sex = []
    Physician_age = []
    Physician_specialty = []
    Physician_location_city = []
    Physician_location_state = []
    Physician_location_zip = []
    Number_of_reviews = []
    Overall_rating = []
    Number_of_ratings = []
    Five_star_ratings = []
    Four_star_ratings = []
    Three_star_ratings = []
    Two_star_ratings = []
    One_star_ratings = []
    Review_text = []
    Review_rating = []
    Review_date = []
    Number_of_helpfuls = []

    # START FUNCTION
    for loc in CITY_LIST:
        city = loc[0] # Loops through each city and state abbreviation.
        state_abb = loc[1]
        for jobtype in JOB_TYPE:
            page_count = 0 # Keeps track of each page, loop, and total pages looped through.
            loop_count = 0
            total_counted = 0
            still_counting = True #
            while still_counting:
                url = f'https://www.healthgrades.com/api3/usearch?where={city}%20{state_abb}' \
                      f'&distances={distance}&sort.provider=bestmatch&what={jobtype}&pageNum={page_count}'

                try:
                    results = requests.post(url).json() # Stop looking when we've hit the final page of search results.
                except:
                    break
                provider_group = results['search']['searchResults']['provider'].get('results')

                if total_counted >= len(provider_group): # Keep track of what search page we're on.
                    page_count += 1
                    total_counted = 0

                for provider in provider_group:
                    total_counted += 1
                    print(total_counted, page_count, loop_count, jobtype)
                    provider_id = provider['providerUrl'][-5:].upper()
                    comments = get_provider_comments(provider_id)
                    comment_count = comments['totalCommentCount']

                    # HANDLE WHETHER WE KEEP THE PROVIDER
                    if loop_count == number_to_list:
                        still_counting = False # If we have enough providers for a certain job type, switch search parameters.
                        continue
                    elif comment_count is None: # Ignore providers with broken or non-standard comment URLs (rare).
                        continue
                    elif comment_count < ignore_below: # Ignore providers with less comments than the desired amount.
                        continue
                    valid_count = 0
                    for comment in comments.get('results'): # Format date and compare against requested number of comments past given date.
                        if datetime.strptime(comment['submittedDate'], "%b %d, %Y") >= datetime.strptime('Jan 01, 2020', "%b %d, %Y"):
                            valid_count += 1
                    if valid_count < 2:
                        continue

                    # Json returned data from other sources does not list star ratings, need to read through some html to grab them from the page.
                    req = requests.get(f"https://www.healthgrades.com{provider['providerUrl']}")
                    soup = BeautifulSoup(req.text, "html.parser")
                    ratings = soup.find_all("td", class_='count')
                    rating_list = [d.text[1:-1] for d in ratings]
                    # Handle json data into neater variables, format strings as necessary.
                    name = provider['displayName']
                    specialty = provider['specialty']
                    location_city = provider['address']['line2'].split(', ')[0]
                    location_state = provider['address']['line2'].split(', ')[1].split(' ')[0]
                    location_zip = str(provider['address']['line2'].split(', ')[1].split(' ')[1])
                    if 'gender' in provider: # Some providers don't have an age listed, this takes that into account.
                        sex = provider['gender']
                    else:
                        sex = 'NaN'
                    if 'age' in provider: # Same issue for age 
                        age = provider['age']
                    else:
                        age = 'NaN'
                    rating_num = provider['surveyUserCount']
                    rating_total = provider['surveyOverallRatingPercent'] / 20 # Ratings return out of 100, break it down into a 0-5 rating.
                    rating_five = rating_list[0]
                    rating_four = rating_list[1]
                    rating_three = rating_list[2]
                    rating_two = rating_list[3]
                    rating_one = rating_list[4]

                    # APPEND DATA TO ARRAYS
                    for comment in comments.get('results'):
                        if len(comment['commentText'].split(' ')) <= 6: # Ignore comments less than 6 words.
                            continue
                        elif comment['commentText'] in Review_text: # Prevent providers with more than one specialty from appearing more than once.
                            continue
                        Physician_name.append(name)
                        Physician_sex.append(sex)
                        Physician_age.append(age)
                        Physician_specialty.append(specialty)
                        Physician_location_city.append(location_city)
                        Physician_location_state.append(location_state)
                        Physician_location_zip.append(location_zip)
                        Number_of_reviews.append(comment_count)
                        Number_of_ratings.append(rating_num)
                        Overall_rating.append(rating_total)
                        Five_star_ratings.append(rating_five)
                        Four_star_ratings.append(rating_four)
                        Three_star_ratings.append(rating_three)
                        Two_star_ratings.append(rating_two)
                        One_star_ratings.append(rating_one)
                        Review_text.append(comment['commentText'])
                        Review_rating.append(f"{comment['overallScore']} out of 5")
                        Review_date.append(comment['submittedDate'])
                        Number_of_helpfuls.append(comment['helpfulCount'])
                    loop_count += 1
    
    # CONVERT ARRAYS INTO PANDAS SERIES
    Physician_Names = pd.Series(Physician_name, dtype="string", name="Physician Name")
    Physician_Sex = pd.Series(Physician_sex, dtype="string", name="Physician Sex")
    Physician_Age = pd.Series(Physician_age, dtype="string", name="Physician Age")
    Physician_Specialty = pd.Series(Physician_specialty, dtype="string", name="Physician Specialty")
    Physician_location_city = pd.Series(Physician_location_city, dtype="string", name="Physician City")
    Physician_location_state = pd.Series(Physician_location_state, dtype="string", name="Physician State")
    Physician_location_zip = pd.Series(Physician_location_zip, dtype="string", name="Physician Zip")
    Number_of_Reviews = pd.Series(Number_of_reviews, dtype="string", name="Number of Reviews")
    Number_of_Ratings = pd.Series(Number_of_ratings, dtype="string", name="Number of Ratings")
    Overall_Rating = pd.Series(Overall_rating, dtype="string", name="Overall Rating")
    Five_Star_Rating = pd.Series(Five_star_ratings, dtype="string", name="Five Star Ratings")
    Four_Star_Rating = pd.Series(Four_star_ratings, dtype="string", name="Four Star Ratings")
    Three_Star_Rating = pd.Series(Three_star_ratings, dtype="string", name="Three Star Ratings")
    Two_Star_Rating = pd.Series(Two_star_ratings, dtype="string", name="Two Star Rating")
    One_Star_Rating = pd.Series(One_star_ratings, dtype="string", name="One Star Rating")
    Review_Dates = pd.Series(Review_date, dtype="string", name="Review Date")
    Review_Texts = pd.Series(Review_text, dtype="string", name="Review Text")
    Review_Rating = pd.Series(Review_rating, dtype="string", name="Review Rating")
    Number_of_Helpfuls = pd.Series(Number_of_helpfuls, dtype="string", name="Number of Helpfuls")

    # CONCATENATE SERIES INTO SINGLE DATAFRAME
    Example_reviews = pd.concat(
        [Physician_Names, Physician_Sex, Physician_Age, Physician_Specialty, Physician_location_city,
         Physician_location_state, Physician_location_zip, Number_of_Reviews,
         Number_of_Ratings, Overall_Rating, Five_Star_Rating, Four_Star_Rating, Three_Star_Rating, Two_Star_Rating,
         One_Star_Rating, Review_Dates, Review_Texts, Review_Rating, Number_of_Helpfuls], axis=1)

    print(Example_reviews)
    Example_reviews.to_csv(r'Example_reviews.csv', index=False, header=True)


# Example code for 50 specilaties per provider type, a 50 mile radius # around each city, and minimum of 25 comments per doctor

## WARNING: RUN TIME MAY BE LONG DEPENDING ON PARAMETERS SELECTED ##

# establish_provider_data(50, 50, 25)